In [1]:
!pip install -q -U bitsandbytes > /dev/null
!pip install -q -U git+https://github.com/huggingface/transformers.git > /dev/null
!pip install -q -U git+https://github.com/huggingface/peft.git > /dev/null
!pip install -q -U git+https://github.com/huggingface/accelerate.git > /dev/null
!pip install -q datasets > /dev/null

^C
ERROR: Operation cancelled by user
^C
ERROR: Operation cancelled by user
^C
ERROR: Operation cancelled by user


In [2]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, default_data_collator, get_linear_schedule_with_warmup
from peft import get_peft_config, get_peft_model, get_peft_model_state_dict, PrefixTuningConfig, TaskType,PromptTuningInit, PromptTuningConfig
from datasets import load_dataset
from torch.utils.data import DataLoader
from tqdm import tqdm
import os

In [3]:
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["CUDA_VISIBLE_DEVICES"] = "3"
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
model_name_or_path = "t5-large"
tokenizer_name_or_path = "t5-large"

text_column = "sentence"
target_column = "built_text"
max_length = 128
lr = 1e-2
num_epochs = 5
batch_size = 8

In [4]:
device

'cuda'

In [5]:
from datasets import load_dataset

dataset = load_dataset("financial_phrasebank",
                       "sentences_allagree")

Dataset financial_phrasebank downloaded and prepared to /home/kamal/.cache/huggingface/datasets/financial_phrasebank/sentences_allagree/1.0.0/550bde12e6c30e2674da973a55f57edde5181d53f5a5a34c1531c53f93b7e141. Subsequent calls will reuse this data.


100%|█████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 654.85it/s]


In [6]:
dataset['train'][0]

{'sentence': 'According to Gran , the company has no plans to move all production to Russia , although that is where the company is growing .',
 'label': 1}

In [7]:
def text_to_table(data):
  """Returns the text into a table format as shown in
  the example below. The column
    Parameter:
      text(str): A sentence

    Returns:
      table_text:
      less_than_6:
      sent_1
      more_than_6:
      sent_2
  """
  text = data['sentence']
  text_splits = text.split(',')

  less_6 = []
  more_6 = []

  for ind, txt in enumerate(text_splits):
    txt = txt.strip()
    snip_len = len(txt.split(' '))

    if snip_len > 6:
      more_6.append(txt)
    else:
      less_6.append(txt)

  built_data = {'less_6':less_6,
                'more_6':more_6}

  data['built_text'] = str(built_data)

  return data

In [8]:
new_dataset = dataset.map(text_to_table)

In [9]:
for text in new_dataset['train']['built_text'][0:2]:
  print(text)

{'less_6': ['According to Gran'], 'more_6': ['the company has no plans to move all production to Russia', 'although that is where the company is growing .']}
{'less_6': ['For the last quarter of 2010'], 'more_6': ["Componenta 's net sales doubled to EUR131m from EUR76m for the same period a year earlier", 'while it moved to a zero pre-tax profit from a pre-tax loss of EUR7m .']}


In [10]:
new_dataset = new_dataset["train"].train_test_split(test_size=0.1)

new_dataset["validation"] = new_dataset["test"]

del new_dataset["test"]

In [11]:
new_dataset

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'built_text'],
        num_rows: 2037
    })
    validation: Dataset({
        features: ['sentence', 'label', 'built_text'],
        num_rows: 227
    })
})

In [1]:
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)


def preprocess_function(examples):
    inputs = examples[text_column]
    targets = examples[target_column]
    model_inputs = tokenizer(inputs,
                             max_length=max_length,
                             padding="max_length",
                             truncation=True,
                             return_tensors="pt")
    labels = tokenizer(targets,
                       max_length=max_length,
                       padding="max_length",
                       truncation=True,
                       return_tensors="pt")

    labels = labels["input_ids"]

    labels[labels == tokenizer.pad_token_id] = -100

    model_inputs["labels"] = labels

    return model_inputs

NameError: name 'AutoTokenizer' is not defined

In [13]:
processed_datasets = new_dataset.map(
    preprocess_function,
    batched=True,
    num_proc=1,
    remove_columns=new_dataset["train"].column_names,
    load_from_cache_file=False,
    desc="Running tokenizer on dataset",
)

In [14]:
processed_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 2037
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 227
    })
})

In [15]:
train_dataset = processed_datasets["train"]
eval_dataset = processed_datasets["validation"]

In [16]:
train_dataloader = DataLoader(train_dataset,
                              shuffle=True,
                              collate_fn=default_data_collator,
                              batch_size=batch_size,
                              pin_memory=True)
eval_dataloader = DataLoader(eval_dataset,
                             collate_fn=default_data_collator,
                             batch_size=batch_size,
                             pin_memory=True)

In [17]:
peft_config = PrefixTuningConfig(task_type=TaskType.SEQ_2_SEQ_LM,
                                 inference_mode=False,
                                 num_virtual_tokens=20)

model = AutoModelForSeq2SeqLM.from_pretrained(model_name_or_path)
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

trainable params: 983040 || all params: 738651136 || trainable%: 0.13308583065659835


In [22]:
import torch

optimizer = torch.optim.AdamW(model.parameters(), lr=lr)

lr_scheduler = get_linear_schedule_with_warmup(
optimizer=optimizer,
num_warmup_steps=0,
num_training_steps=(len(train_dataloader) * num_epochs),
)

In [18]:
model.to(device)

PeftModelForSeq2SeqLM(
  (base_model): T5ForConditionalGeneration(
    (shared): Embedding(32128, 1024)
    (encoder): T5Stack(
      (embed_tokens): Embedding(32128, 1024)
      (block): ModuleList(
        (0): T5Block(
          (layer): ModuleList(
            (0): T5LayerSelfAttention(
              (SelfAttention): T5Attention(
                (q): Linear(in_features=1024, out_features=1024, bias=False)
                (k): Linear(in_features=1024, out_features=1024, bias=False)
                (v): Linear(in_features=1024, out_features=1024, bias=False)
                (o): Linear(in_features=1024, out_features=1024, bias=False)
                (relative_attention_bias): Embedding(32, 16)
              )
              (layer_norm): T5LayerNorm()
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (1): T5LayerFF(
              (DenseReluDense): T5DenseActDense(
                (wi): Linear(in_features=1024, out_features=4096, bias=False)
             

In [24]:
!nvidia-smi

Sun Jul 16 07:47:31 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA Graphics...  Off  | 00000000:01:00.0  On |                  N/A |
| 30%   40C    P2    38W / 200W |   8241MiB / 12282MiB |     14%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [20]:
train_dataset[0].items()

dict_items([('input_ids', [9682, 7, 33, 59, 19972, 3, 5, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), ('attention_mask', [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), ('labels', [3, 2, 31, 924, 834, 948, 31, 10, 784, 31, 13898, 4128, 33, 59, 19972, 3, 5, 31, 13679, 3, 31, 3706, 834, 948, 31, 10, 784, 908, 2, 1, -100, -100, -100, -100, -1

In [23]:
for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    for step, batch in enumerate(tqdm(train_dataloader)):
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        total_loss += loss.detach().float()
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()

    model.eval()
    eval_loss = 0
    eval_preds = []

    for step, batch in enumerate(tqdm(eval_dataloader)):
        batch = {k: v.to(device) for k, v in batch.items()}
        with torch.no_grad():
            outputs = model(**batch)
        loss = outputs.loss
        eval_loss += loss.detach().float()
        eval_preds.extend(
            tokenizer.batch_decode(torch.argmax(outputs.logits, -1).detach().cpu().numpy(), skip_special_tokens=True)
        )

    eval_epoch_loss = eval_loss / len(eval_dataloader)
    eval_ppl = torch.exp(eval_epoch_loss)
    train_epoch_loss = total_loss / len(train_dataloader)
    train_ppl = torch.exp(train_epoch_loss)
    print(f"{epoch=}: {train_ppl=} {train_epoch_loss=} {eval_ppl=} {eval_epoch_loss=}")

100%|████████████████████████████████████████████████████████████████████████| 29/29 [00:03<00:00,  7.53it/s]


epoch=0: train_ppl=tensor(1.9938, device='cuda:0') train_epoch_loss=tensor(0.6901, device='cuda:0') eval_ppl=tensor(1.2096, device='cuda:0') eval_epoch_loss=tensor(0.1903, device='cuda:0')


100%|████████████████████████████████████████████████████████████████████████| 29/29 [00:03<00:00,  7.26it/s]


epoch=1: train_ppl=tensor(1.2981, device='cuda:0') train_epoch_loss=tensor(0.2609, device='cuda:0') eval_ppl=tensor(1.1595, device='cuda:0') eval_epoch_loss=tensor(0.1480, device='cuda:0')


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 29/29 [00:04<00:00,  7.17it/s]


epoch=2: train_ppl=tensor(1.2393, device='cuda:0') train_epoch_loss=tensor(0.2145, device='cuda:0') eval_ppl=tensor(1.1370, device='cuda:0') eval_epoch_loss=tensor(0.1284, device='cuda:0')


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 29/29 [00:04<00:00,  7.02it/s]


epoch=3: train_ppl=tensor(1.2156, device='cuda:0') train_epoch_loss=tensor(0.1952, device='cuda:0') eval_ppl=tensor(1.1259, device='cuda:0') eval_epoch_loss=tensor(0.1186, device='cuda:0')


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 29/29 [00:04<00:00,  7.13it/s]

epoch=4: train_ppl=tensor(1.2033, device='cuda:0') train_epoch_loss=tensor(0.1851, device='cuda:0') eval_ppl=tensor(1.1226, device='cuda:0') eval_epoch_loss=tensor(0.1156, device='cuda:0')


In [58]:
new_dataset['validation'][0:4]

{'sentence': ["PCS Digital Guatemala has been using Tecnomen 's prepaid system in Telgua fixed networks as well as code division multiple access , or CDMA , and global system for mobile communication , or GSM , since 2003 .",
  'Customers include hotels and restaurants as well as wholesalers and some retailers .',
  "`` UPM 's deliveries increased during the third quarter by 4 percent , and the efficiency of operations improved , '' Chief Executive Jussi Pesonen said .",
  'Aspocomp Group , headquartered in Helsinki , Finland , develops interconnection solutions for the electronics industry .'],
 'label': [1, 1, 2, 1],
 'built_text': ['{\'less_6\': [\'or CDMA\', \'and global system for mobile communication\', \'or GSM\', \'since 2003 .\'], \'more_6\': ["PCS Digital Guatemala has been using Tecnomen \'s prepaid system in Telgua fixed networks as well as code division multiple access"]}',
  "{'less_6': [], 'more_6': ['Customers include hotels and restaurants as well as wholesalers and so

In [51]:
print(eval_preds[0])

'less_6': [],PC CDMA', 'and global system for mobile communication', 'or GSM, 'Tec 2003.'],'more_6': ['PCS Digital Guatemala has been using Tecnomen's prepaid system in Telgua fixed networks and well as G division multiple access"]' 


In [52]:
print(eval_preds[1])

'less_6': [],'more_6': ['Customers include hotels and restaurants as well as somers and some retailers.']                             


In [57]:
print(eval_preds[2])

'less_6': ['' the efficiency of operations improved'],'more_6': [' UPM's deliveries increased during the third quarter by 4 percent", 'CC Executive Jussi Pesonen said.]en


In [59]:
print(eval_preds[3])

'less_6': [],headquarteredspocomp Group', 'headquartered in Helsinki', 'Finland','more_6': ['develops interconnection solutions for the electronics industry.']'  A A A A A A A A A A A A A A  A A A A A


In [60]:
print(eval_preds[4])

'less_6': [],'more_6': ["Estonia's Agriculture Minister Helir-Valdor Seeder is in Finland on a two-day visit", 'in the course of which he will meet with his Finnish counterpart Sirkka-Liisa Anttila.']]


In [43]:
new_dataset

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'built_text'],
        num_rows: 2037
    })
    validation: Dataset({
        features: ['sentence', 'label', 'built_text'],
        num_rows: 227
    })
})

In [62]:
correct = 0
total = 0
for pred, true in zip(eval_preds, new_dataset["validation"]["built_text"]):
    if pred.strip() == true.strip():
        correct += 1
    total += 1
accuracy = correct / total * 100

print(f"{accuracy=} % on the evaluation dataset")

print(f"{eval_preds[:10]=}")

print(f"{new_dataset['validation']['built_text'][:10]=}")


accuracy=0.0 % on the evaluation dataset
eval_preds[:10]=['\'less_6\': [],PC CDMA\', \'and global system for mobile communication\', \'or GSM, \'Tec 2003.\'],\'more_6\': [\'PCS Digital Guatemala has been using Tecnomen\'s prepaid system in Telgua fixed networks and well as G division multiple access"]\' ', "'less_6': [],'more_6': ['Customers include hotels and restaurants as well as somers and some retailers.']                             ", '\'less_6\': [\'\' the efficiency of operations improved\'],\'more_6\': [\' UPM\'s deliveries increased during the third quarter by 4 percent", \'CC Executive Jussi Pesonen said.]en', "'less_6': [],headquarteredspocomp Group', 'headquartered in Helsinki', 'Finland','more_6': ['develops interconnection solutions for the electronics industry.']'  A A A A A A A A A A A A A A  A A A A A", '\'less_6\': [],\'more_6\': ["Estonia\'s Agriculture Minister Helir-Valdor Seeder is in Finland on a two-day visit", \'in the course of which he will meet with his Fi

In [63]:
from huggingface_hub import notebook_login

notebook_login()

In [64]:
peft_model_id = "Kamaljp/t5-large_text_building"

model.push_to_hub(peft_model_id,
                  use_auth_token=True)

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

adapter_model.bin:   0%|          | 0.00/3.93M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Kamaljp/t5-large_text_building/commit/954ebca5657f5e6a3fbb17181d62100d1b52a1fb', commit_message='Upload model', commit_description='', oid='954ebca5657f5e6a3fbb17181d62100d1b52a1fb', pr_url=None, pr_revision=None, pr_num=None)

In [65]:
from peft import PeftModel, PeftConfig

config = PeftConfig.from_pretrained(peft_model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(config.base_model_name_or_path)
model = PeftModel.from_pretrained(model, peft_model_id)

In [66]:
inputs = tokenizer(
    "The Indian beer market, made up 24.41 million liters in January , a rise of 18 percent.",
    return_tensors="pt",
)

In [68]:
model.to(device)

with torch.no_grad():
    inputs = {k: v.to(device) for k, v in inputs.items()}
    outputs = model.generate(input_ids=inputs["input_ids"], max_new_tokens=128)
    print(tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True))

["'less_6': [],'more_6': ['The Indian beer market, made 24.41 million liters in January', 'a rise of 18 percent.']"]


In [69]:
def generate_output(text):
  inputs = tokenizer(text,return_tensors='pt')
  with torch.no_grad():
    inputs = {k: v.to(device) for k, v in inputs.items()}
    outputs = model.generate(input_ids = inputs['input_ids'],
                             max_new_tokens = 128)
    gen_decode = tokenizer.batch_decode(outputs.detach().cpu().numpy(),
                                        skip_special_tokens = True)

  return gen_decode

Working on Causal Language modelling

In [ ]:
device = "cuda"
model_name_or_path = "bigscience/bloomz-560m"
tokenizer_name_or_path = "bigscience/bloomz-560m"
peft_config = PromptTuningConfig(
    task_type=TaskType.CAUSAL_LM,
    prompt_tuning_init=PromptTuningInit.TEXT,
    num_virtual_tokens=8,
    prompt_tuning_init_text="Classify if the tweet is a complaint or not:",
    tokenizer_name_or_path=model_name_or_path,
)

dataset_name = "twitter_complaints"
checkpoint_name = f"{dataset_name}_{model_name_or_path}_{peft_config.peft_type}_{peft_config.task_type}_v1.pt".replace(
    "/", "_"
)
text_column = "Tweet text"
label_column = "text_label"
max_length = 64
lr = 3e-2
num_epochs = 50
batch_size = 8

In [ ]:
dataset = load_dataset("ought/raft", dataset_name)
dataset["train"][0]

Extracting data files:   0%|          | 0/11 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/50 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3399 [00:00<?, ? examples/s]

Dataset raft downloaded and prepared to /root/.cache/huggingface/datasets/ought___raft/twitter_complaints/1.1.0/79c4de1312c1e3730043f7db07179c914f48403101f7124e2fe336f6f54d9f84. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

{'Tweet text': '@HMRCcustomers No this is my first job', 'ID': 0, 'Label': 2}

In [ ]:
dataset["train"].feature

Dataset({
    features: ['Tweet text', 'ID', 'Label'],
    num_rows: 50
})

In [ ]:
dataset["train"].features["Label"].names

['Unlabeled', 'complaint', 'no complaint']

In [ ]:
classes = [k.replace("_", " ") for k in dataset["train"].features["Label"].names]
dataset = dataset.map(
    lambda x: {"text_label": [classes[label] for label in x["Label"]]},
    batched=True,
    num_proc=1,
)
dataset["train"][0]

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

Map:   0%|          | 0/3399 [00:00<?, ? examples/s]

{'Tweet text': '@HMRCcustomers No this is my first job',
 'ID': 0,
 'Label': 2,
 'text_label': 'no complaint'}

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)

In [ ]:
tokenizer.eos_token_id

2

In [ ]:
if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id

target_max_length = max([len(tokenizer(class_label)["input_ids"]) for class_label in classes])

print(target_max_length)


3


In [ ]:
import torch

def preprocess_function(examples):
    batch_size = len(examples[text_column])
    inputs = [f"{text_column} : {x} Label : " for x in examples[text_column]]
    targets = [str(x) for x in examples[label_column]]
    model_inputs = tokenizer(inputs)
    labels = tokenizer(targets)
    for i in range(batch_size):
        sample_input_ids = model_inputs["input_ids"][i]
        label_input_ids = labels["input_ids"][i] + [tokenizer.pad_token_id]
        # print(i, sample_input_ids, label_input_ids)
        model_inputs["input_ids"][i] = sample_input_ids + label_input_ids
        labels["input_ids"][i] = [-100] * len(sample_input_ids) + label_input_ids
        model_inputs["attention_mask"][i] = [1] * len(model_inputs["input_ids"][i])
    # print(model_inputs)
    for i in range(batch_size):
        sample_input_ids = model_inputs["input_ids"][i]
        label_input_ids = labels["input_ids"][i]
        model_inputs["input_ids"][i] = [tokenizer.pad_token_id] * (
            max_length - len(sample_input_ids)
        ) + sample_input_ids
        model_inputs["attention_mask"][i] = [0] * (max_length - len(sample_input_ids)) + model_inputs[
            "attention_mask"
        ][i]
        labels["input_ids"][i] = [-100] * (max_length - len(sample_input_ids)) + label_input_ids
        model_inputs["input_ids"][i] = torch.tensor(model_inputs["input_ids"][i][:max_length])
        model_inputs["attention_mask"][i] = torch.tensor(model_inputs["attention_mask"][i][:max_length])
        labels["input_ids"][i] = torch.tensor(labels["input_ids"][i][:max_length])
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
processed_datasets = dataset.map(
    preprocess_function,
    batched=True,
    num_proc=1,
    remove_columns=dataset["train"].column_names,
    load_from_cache_file=False,
    desc="Running tokenizer on dataset",
)

Running tokenizer on dataset:   0%|          | 0/50 [00:00<?, ? examples/s]

Running tokenizer on dataset:   0%|          | 0/3399 [00:00<?, ? examples/s]

In [ ]:
train_dataset = processed_datasets["train"]
eval_dataset = processed_datasets["train"]


train_dataloader = DataLoader(train_dataset, shuffle=True,
                              collate_fn=default_data_collator,
                              batch_size=batch_size,
                              pin_memory=True)
eval_dataloader = DataLoader(eval_dataset,
                             collate_fn=default_data_collator,
                             batch_size=batch_size,
                             pin_memory=True)

In [ ]:
from transformers import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained(model_name_or_path)

model = get_peft_model(model, peft_config)

print(model.print_trainable_parameters())

trainable params: 8,192 || all params: 559,222,784 || trainable%: 0.0014648902430985358
None


In [ ]:
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)

lr_scheduler = get_linear_schedule_with_warmup(
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=(len(train_dataloader) * num_epochs),
)

In [ ]:
model = model.to(device)

In [ ]:
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for step, batch in enumerate(tqdm(train_dataloader)):
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        total_loss += loss.detach().float()
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()

    model.eval()
    eval_loss = 0
    eval_preds = []
    for step, batch in enumerate(tqdm(eval_dataloader)):
        batch = {k: v.to(device) for k, v in batch.items()}
        with torch.no_grad():
            outputs = model(**batch)
        loss = outputs.loss
        eval_loss += loss.detach().float()
        eval_preds.extend(
            tokenizer.batch_decode(torch.argmax(outputs.logits, -1).detach().cpu().numpy(), skip_special_tokens=True)
        )

    eval_epoch_loss = eval_loss / len(eval_dataloader)
    eval_ppl = torch.exp(eval_epoch_loss)
    train_epoch_loss = total_loss / len(train_dataloader)
    train_ppl = torch.exp(train_epoch_loss)
    print(f"{epoch=}: {train_ppl=} {train_epoch_loss=} {eval_ppl=} {eval_epoch_loss=}")

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
peft_model_id = "Kamaljp/bloomz-560m_PROMPT_TUNING_CAUSAL_LM"
model.push_to_hub("Kamaljp/bloomz-560m_PROMPT_TUNING_CAUSAL_LM", use_auth_token=True)

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

adapter_model.bin:   0%|          | 0.00/33.5k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Kamaljp/bloomz-560m_PROMPT_TUNING_CAUSAL_LM/commit/7a7a82d99c176ec88a2ed91dbef9004e269ce9b5', commit_message='Upload model', commit_description='', oid='7a7a82d99c176ec88a2ed91dbef9004e269ce9b5', pr_url=None, pr_revision=None, pr_num=None)